In [30]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = ""
api_key = os.getenv('MISTRAL_API_KEY')

In [31]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

model = init_chat_model("mistral-large-latest", model_provider="mistralai")
search = TavilySearchResults(max_results=2)
tools = [search] # All tools in list

#bind tools with model
model_with_tools = model.bind_tools(tools)

In [ ]:
#Invoke model with message
response = model_with_tools.invoke([HumanMessage(content="Hi there!")])

print(f"Response by Model- ContentString : {response.content}")
print(f"Response by Tavily- ToolCalls : {response.tool_calls}")

In [37]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Delhi?")])

print(f"Response by Model- ContentString: {response.content}")
print(f"Response by Tavily- ToolCalls : {response.tool_calls}")
# Model Response to call tools
#This isn't calling that tool yet - it's just telling us to. 
#In order to actually call it, we'll want to create our agent.

Response by Model- ContentString: 
Response by Tavily- ToolCalls : [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Delhi'}, 'id': 'NFn94EtII', 'type': 'tool_call'}]


In [44]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)
# Use the agent
response = agent_executor.invoke({"messages": [HumanMessage(content="Hi Model!")]})

response["messages"]

[HumanMessage(content='Hi Model!', additional_kwargs={}, response_metadata={}, id='5cca52ca-dece-4d83-8ed2-02002c6a253b'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 110, 'total_tokens': 119, 'completion_tokens': 9}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-5302c2a2-94be-4368-ba8e-ff64600bb326-0', usage_metadata={'input_tokens': 110, 'output_tokens': 9, 'total_tokens': 119})]

In [ ]:
#Streaming

for step in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi there!")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi there!
================================== Ai Message ==================================

Hello! How can I assist you today? If you have any questions or need help with something specific, feel free to let me know!


In [41]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="what's the weather where in Delhi?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

what's the weather where in Delhi?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (Cp3L1ULqV)
 Call ID: Cp3L1ULqV
  Args:
    query: weather in Delhi
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "[PDF] Dated: 23 March 2025 Time of Issue: 1815 Hrs. IST WEATHER ...", "url": "https://mausam.imd.gov.in/newdelhi/mcdata/delhi_forecast.pdf", "content": "IN THE MORNING. MAINLY CLEAR SKY.STRONG SURFACE WIND (SPEED 20-30) KMPH DURING THE DAY. West Delhi LIKELY LIKELY LIKELY LIKELY LIKELY MAINLY CLEAR SKY. SMOG/MIST IN THE MORNING. MAINLY CLEAR SKY. SMOG/SHALLOW FOG IN THE MORNING. MAINLY CLEAR SKY. SMOG/MIST IN THE NIGHT. SMOG/SHALLOW FOG IN THE MORNING. MAINLY CLEAR SKY. SMOG/MIST IN THE NIGHT. SMOG/MIST IN THE MORNING. MAINLY CLEAR SKY.STRONG SURFACE WIND (SPE

In [53]:
# For Memory Persistence, keeping conversation history

from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

#Agent will have memory now
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [54]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im joy!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Joy! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 111, 'total_tokens': 121, 'completion_tokens': 10}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-0a855c6f-6716-4c16-8262-80a99749473c-0', usage_metadata={'input_tokens': 111, 'output_tokens': 10, 'total_tokens': 121})]}}
----


In [55]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You said your name is Joy!', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 129, 'total_tokens': 136, 'completion_tokens': 7}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-660ad845-3792-4d15-8734-6ad61baddecb-0', usage_metadata={'input_tokens': 129, 'output_tokens': 7, 'total_tokens': 136})]}}
----
